In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from PMTK.sampling.preferences_sampler import *
from PMTK.pref.preferences import *
from PMTK.utility.utility_solver import *
from PMTK.sampling.subset_samplers import *
from PMTK.utility.subset_finder import *
from PMTK.sampling.gibbs import *
from PMTK.data.film_dataset import *
from PMTK.sampling.decider import *

In [2]:
def theta_better(theta_1, theta_2):
    if additivity(theta_1) != additivity(theta_2):
        return 1 if additivity(theta_1) < additivity(theta_2) else -1
    else:
        if len(theta_1) != len(theta_2):
            return 1 if len(theta_1) < len(theta_2) else -1
        else:
            return 0
        
def additivity(theta):
    return max([len(i) for i in theta])

In [3]:
def get_candidate_iterator(c, bound = np.inf):
    a = list(set([x[0] for x in c if len(x[0]) > 0] + [x[1] for x in c if len(x[1]) > 0]))
    a = sorted(a, key = lambda x:len(x))
    iterators = {}
    for x in a:
        for k in range(1,min(len(x)+1, bound)):
            iterators[k] = iterators.get(k,[]) + [itertools.combinations(x, k)]
    for k in iterators:
        random.shuffle(iterators[k])
    return iterators

def dfs_thetas_r(preferences, theta, theta_mins = [], banned = None, bann_opt = True):
    if not banned:
        banned = []
    c = get_connivent(theta, preferences)
    print("Theta = ", theta, end = "\t")
    print("c = ", c, end = "\t")
    print("Banned = ", banned)
    
    if c == None:
        kernels = [theta]
        #kernels += get_kernels(prf, theta)
        for theta in kernels:
            if theta in theta_mins:
                continue
            if len(theta_mins) == 0:
                theta_mins.append(theta)
                return additivity(theta_mins[0]), len(theta_mins[0])
            if theta_better(theta, theta_mins[0]) == 1:
                theta_mins.clear()
                theta_mins.append(theta)
            elif theta_better(theta, theta_mins[0]) == 0:
                theta_mins.append(theta)
                
        return additivity(theta_mins[0]), len(theta_mins[0])
    
    its = get_candidate_iterator(c)
    b = list(banned)
    for k in its:
        for candidates in its[k]:
            for c in candidates:
                if len(theta_mins) > 0 and k > additivity(theta_mins[0]):
                    #print("Cutting over", k, " in:", theta , end = " ")
                    break
                if c in theta or c in b:
                    continue
                n_theta = theta + [c]
                if len(theta_mins) > 0 and theta_better(n_theta, theta_mins[0]) == -1:
                    #print("Cutting", c , " in:", theta, end = " ")
                    continue
                print("Trying candidate ", c)
                dfs_thetas_r(preferences, n_theta, theta_mins, banned = b)
                b.append(c)
    return True

def get_min_thetas(preferences, initial_theta = None, bann_opt = True):
    representant = None
    theta_mins = []
    stats = []
    if not initial_theta:
        initial_theta = [EMPTY_SET]
    dfs_thetas_r(preferences, initial_theta, theta_mins = theta_mins, bann_opt = bann_opt)
    return theta_mins, stats


In [15]:
n_items = 6
density = 0.2
items = np.arange(n_items)
prf = sample_preferences_from_order(items, density*(2**(n_items+1)), indifference_rate= 0)

In [16]:
t_mins, _ = get_min_thetas(prf)

Theta =  []	c =  [((2, 4), ())]	Banned =  []
Trying candidate  (2,)
Theta =  [(2,)]	c =  [((1, 3), ())]	Banned =  []
Trying candidate  (1,)
Theta =  [(2,), (1,)]	c =  [((1, 3, 5), (1, 3))]	Banned =  []
Trying candidate  (3,)
Theta =  [(2,), (1,), (3,)]	c =  [((1, 4, 5), (1, 2, 4, 5)), ((2, 4), (1, 3)), ((1, 3), ())]	Banned =  []
Trying candidate  (4,)
Theta =  [(2,), (1,), (3,), (4,)]	c =  [((1, 3, 5), (1, 3))]	Banned =  []
Trying candidate  (5,)
Theta =  [(2,), (1,), (3,), (4,), (5,)]	c =  [((0, 1, 3, 5), (0, 2, 3, 4, 5)), ((1, 3, 5), (2, 4)), ((2, 4), (1, 3)), ((1, 3), (1, 2, 4, 5)), ((1, 3), ())]	Banned =  []
Trying candidate  (0,)
Theta =  [(2,), (1,), (3,), (4,), (5,), (0,)]	c =  [((0, 1, 3, 5), (0, 2, 3, 4, 5)), ((1, 3, 5), (2, 4)), ((2, 4), (1, 3)), ((1, 3), (1, 2, 4, 5)), ((1, 3), ())]	Banned =  []
Trying candidate  (1, 3)
Theta =  [(2,), (1,), (3,), (4,), (5,), (0,), (1, 3)]	c =  None	Banned =  []
Trying candidate  (0, 1)
Theta =  [(2,), (1,), (3,), (4,), (5,), (0,), (0, 1)]	c

In [17]:
t2 = []
for k in t_mins:
    t2.append(tuple(k))
t2 = list(set(t2))
len(t2)

35

In [18]:
t2

[((4,), (1, 3), (5,), (2, 5), (0,)),
 ((4,), (3,), (1, 5), (1, 2), (0, 3)),
 ((4,), (1,), (3, 5), (2, 5), (0,)),
 ((4,), (1,), (3, 5), (2, 5), (0, 3)),
 ((4,), (1, 3), (1, 5), (1, 2), (0, 3)),
 ((4,), (1,), (1, 5), (1, 2), (0, 3)),
 ((4,), (1,), (5,), (2, 5), (0, 5)),
 ((4,), (1,), (3, 5), (1, 2), (0, 1)),
 ((4,), (3,), (1, 5), (2, 5), (0,)),
 ((4,), (1, 3), (1, 5), (2, 5), (0, 5)),
 ((4,), (3,), (5,), (2, 5), (0, 1)),
 ((4,), (1, 3), (1, 5), (1, 2), (0, 5)),
 ((4,), (1, 3), (1, 5), (1, 2), (0,)),
 ((4,), (3,), (1, 5), (1, 2), (0, 1)),
 ((4,), (1,), (5,), (2, 5), (0,)),
 ((4,), (3,), (5,), (0, 5), (2, 5)),
 ((4,), (1,), (3, 5), (2, 5), (0, 5)),
 ((4,), (1, 3), (1, 5), (2, 5), (0,)),
 ((4,), (3,), (5,), (0, 3), (2, 5)),
 ((4,), (3,), (1, 5), (2, 5), (0, 5)),
 ((4,), (1,), (1, 5), (2, 5), (0, 5)),
 ((4,), (3,), (1, 5), (2, 5), (0, 3)),
 ((4,), (3,), (5,), (0,), (2, 5)),
 ((4,), (1, 3), (5,), (2, 5), (0, 5)),
 ((4,), (1,), (1, 5), (1, 2), (0,)),
 ((4,), (1, 3), (1, 5), (2, 5), (0, 3)),
 (

In [19]:
t_mins, _ = get_min_thetas(prf)

Theta =  []	c =  [((2, 4), ())]	Banned =  []
Trying candidate  (2,)
Theta =  [(2,)]	c =  [((1, 3), ())]	Banned =  []
Trying candidate  (1,)
Theta =  [(2,), (1,)]	c =  [((1, 3, 5), (1, 3))]	Banned =  []
Trying candidate  (3,)
Theta =  [(2,), (1,), (3,)]	c =  [((1, 4, 5), (1, 2, 4, 5)), ((2, 4), (1, 3)), ((1, 3), ())]	Banned =  []
Trying candidate  (4,)
Theta =  [(2,), (1,), (3,), (4,)]	c =  [((1, 3, 5), (1, 3))]	Banned =  []
Trying candidate  (5,)
Theta =  [(2,), (1,), (3,), (4,), (5,)]	c =  [((0, 1, 3, 5), (0, 2, 3, 4, 5)), ((1, 3, 5), (2, 4)), ((2, 4), (1, 3)), ((1, 3), (1, 2, 4, 5)), ((1, 3), ())]	Banned =  []
Trying candidate  (0,)
Theta =  [(2,), (1,), (3,), (4,), (5,), (0,)]	c =  [((0, 1, 3, 5), (0, 2, 3, 4, 5)), ((1, 3, 5), (2, 4)), ((2, 4), (1, 3)), ((1, 3), (1, 2, 4, 5)), ((1, 3), ())]	Banned =  []
Trying candidate  (0, 2)
Theta =  [(2,), (1,), (3,), (4,), (5,), (0,), (0, 2)]	c =  None	Banned =  []
Trying candidate  (0, 3)
Theta =  [(2,), (1,), (3,), (4,), (5,), (0,), (0, 3)]	c

In [20]:
t2 = []
for k in t_mins:
    t2.append(tuple(k))
t2 = list(set(t2))
len(t2)

35

In [21]:
t2

[((4,), (1, 3), (5,), (2, 5), (0,)),
 ((4,), (3,), (1, 5), (1, 2), (0, 3)),
 ((4,), (1,), (3, 5), (2, 5), (0,)),
 ((4,), (1,), (3, 5), (2, 5), (0, 3)),
 ((4,), (1, 3), (1, 5), (1, 2), (0, 3)),
 ((4,), (1,), (1, 5), (1, 2), (0, 3)),
 ((4,), (1,), (5,), (2, 5), (0, 5)),
 ((4,), (1,), (3, 5), (1, 2), (0, 1)),
 ((4,), (3,), (1, 5), (2, 5), (0,)),
 ((4,), (1, 3), (1, 5), (2, 5), (0, 5)),
 ((4,), (3,), (5,), (2, 5), (0, 1)),
 ((4,), (1, 3), (1, 5), (1, 2), (0, 5)),
 ((4,), (1, 3), (1, 5), (1, 2), (0,)),
 ((4,), (3,), (1, 5), (1, 2), (0, 1)),
 ((4,), (1,), (5,), (2, 5), (0,)),
 ((4,), (3,), (5,), (0, 5), (2, 5)),
 ((4,), (1,), (3, 5), (2, 5), (0, 5)),
 ((4,), (1, 3), (1, 5), (2, 5), (0,)),
 ((4,), (3,), (5,), (0, 3), (2, 5)),
 ((4,), (3,), (1, 5), (2, 5), (0, 5)),
 ((4,), (1,), (1, 5), (2, 5), (0, 5)),
 ((4,), (3,), (1, 5), (2, 5), (0, 3)),
 ((4,), (3,), (5,), (0,), (2, 5)),
 ((4,), (1, 3), (5,), (2, 5), (0, 5)),
 ((4,), (1,), (1, 5), (1, 2), (0,)),
 ((4,), (1, 3), (1, 5), (2, 5), (0, 3)),
 (